# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [65]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [66]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [67]:
# 3) seu código aqui
# Colunas que interessam: ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    #'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
    
df_sinasc = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
                    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df_sinasc.isna().sum().sort_values(ascending=False)

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

In [68]:
df_sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [69]:
print(f"{df_sinasc['APGAR5'].isna().sum()} APGAR5 vazios")
print(df_sinasc.shape[0])
df_sinasc.dropna(subset=['APGAR5'], inplace=True)
print(f"{df_sinasc['APGAR5'].isna().sum()} APGAR5 vazios")
print(df_sinasc.shape[0])

103 APGAR5 vazios
27028
0 APGAR5 vazios
26925


C:\Users\Bill_\AppData\Local\Temp\ipykernel_1344\420881245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc.dropna(subset=['APGAR5'], inplace=True)


In [70]:
# 5) seu código aqui
#df_sinasc['ESTCIVMAE'] = df_sinasc['ESTCIVMAE'].replace(9.0, pd.NA)
df_sinasc['ESTCIVMAE'] = df_sinasc['ESTCIVMAE'].mask(df_sinasc['ESTCIVMAE']==9.0, pd.NA)
df_sinasc

C:\Users\Bill_\AppData\Local\Temp\ipykernel_1344\2554047958.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc['ESTCIVMAE'] = df_sinasc['ESTCIVMAE'].mask(df_sinasc['ESTCIVMAE']==9.0, pd.NA)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [71]:
print(f"{df_sinasc['ESTCIVMAE'].isna().sum()} ESTCIVMAE vazios")

459 ESTCIVMAE vazios


In [72]:
print(f"{df_sinasc['CONSULTAS'].isna().sum()} CONSULTAS vazios")

0 CONSULTAS vazios


In [73]:
df_sinasc['CONSULTAS'] = df_sinasc['CONSULTAS'].mask(df_sinasc['CONSULTAS']==9.0, pd.NA)
print(f"{df_sinasc['CONSULTAS'].isna().sum()} CONSULTAS vazios")

50 CONSULTAS vazios


C:\Users\Bill_\AppData\Local\Temp\ipykernel_1344\390097343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc['CONSULTAS'] = df_sinasc['CONSULTAS'].mask(df_sinasc['CONSULTAS']==9.0, pd.NA)


In [74]:
# 6) Seu código aqui
df_sinasc['QTDFILVIVO'].isna().sum()
df_sinasc['QTDFILVIVO'] = df_sinasc['QTDFILVIVO'].fillna(0)
df_sinasc

C:\Users\Bill_\AppData\Local\Temp\ipykernel_1344\3988474954.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc['QTDFILVIVO'] = df_sinasc['QTDFILVIVO'].fillna(0)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4.0,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4.0,9.0


In [75]:
# 7) seu código aqui
print(f"{df_sinasc['ESCMAE'].isna().sum()} ESCMAE vazios")
print(f"{df_sinasc['LOCNASC'].isna().sum()} LOCNASC vazios")
print(f"{df_sinasc['IDADEMAE'].isna().sum()} IDADEMAE vazios")
print(f"{df_sinasc['GESTACAO'].isna().sum()} GESTACAO vazios")
print(f"{df_sinasc['GRAVIDEZ'].isna().sum()} GRAVIDEZ vazios")

310 ESCMAE vazios
0 LOCNASC vazios
0 IDADEMAE vazios
1216 GESTACAO vazios
76 GRAVIDEZ vazios


In [76]:
df_sinasc['LOCNASC'].unique()

array([1, 4, 2, 3], dtype=int64)

In [77]:
df_sinasc['LOCNASC'].value_counts()

1    26824
3       40
2       35
4       26
Name: LOCNASC, dtype: int64

In [78]:
df_sinasc['IDADEMAE'].unique()

array([19, 29, 37, 30, 33, 27, 20, 31, 21, 18, 24, 22, 25, 26, 34, 23, 28,
       14, 17, 40, 16, 35, 39, 38, 32, 36, 41, 15, 42, 43, 12, 13, 47, 45,
       44, 11, 53, 46, 51, 52], dtype=int64)

Como foi usado o isna() nas colunas LOCNASC e IDADEMAE retornando o valor 0 e não tem nenhum valor que aparentemente possa significar o valor faltante, pode-se presumir que realmente não tem valores faltantes. Assim foi verificado os valores que tiveram retorno de 0 no isna() em conjunto com o sum(), agora será visto as demais colunas.

In [79]:
df_sinasc['ESCMAE'].unique()

array(['8 a 11 anos', '12 anos ou mais', '1 a 3 anos', '4 a 7 anos', nan,
       'Nenhuma'], dtype=object)

In [80]:
proporcao_ESCMAE = round(df_sinasc['ESCMAE'].isna().sum()/df_sinasc.shape[0],3)
proporcao_ESCMAE


0.012

In [81]:
df_sinasc['GRAVIDEZ'].unique()

array(['Única', 'Dupla', nan, 'Tríplice e mais'], dtype=object)

In [82]:
proporcao_GRAVIDEZ = round(df_sinasc['GRAVIDEZ'].isna().sum()/df_sinasc.shape[0],3)
proporcao_GRAVIDEZ

0.003

In [83]:
df_sinasc['GESTACAO'].unique()

array(['37 a 41 semanas', '42 semanas e mais', '32 a 36 semanas',
       '22 a 27 semanas', '28 a 31 semanas', nan, 'Menos de 22 semanas'],
      dtype=object)

In [84]:
df_sinasc.dropna(subset=['ESCMAE', 'GRAVIDEZ'], inplace=True)
print(f"{df_sinasc['ESCMAE'].isna().sum()} GESTACAO vazios")
print(f"{df_sinasc['GRAVIDEZ'].isna().sum()} GRAVIDEZ vazios")

0 GESTACAO vazios


C:\Users\Bill_\AppData\Local\Temp\ipykernel_1344\1056212026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc.dropna(subset=['ESCMAE', 'GRAVIDEZ'], inplace=True)


0 GRAVIDEZ vazios


Foi removido os valores vazios de ESCMAE e GRAVIDEZ, pois ambos tinham proporção baixa de valores vazios em comparação ao total e os vazios da variável de interesse também foi removidas.

Para fins de estudo, os valores vazios de GESTACAO vão ser alterados para string 'Não preenchido'.

In [85]:
df_sinasc['GESTACAO'].value_counts()

37 a 41 semanas        22243
32 a 36 semanas         2189
42 semanas e mais        693
28 a 31 semanas          158
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [86]:
proporcao_GESTACAO = round(df_sinasc['GESTACAO'].isna().sum()/df_sinasc.shape[0],3)
proporcao_GESTACAO

0.044

In [87]:
df_sinasc['GESTACAO'].value_counts().sum()
proporcao_37a41 = 22243/df_sinasc['GESTACAO'].value_counts().sum()
proporcao_37a41


0.8765714285714286

In [88]:
df_sinasc['GESTACAO'] = df_sinasc['GESTACAO'].fillna('Não preenchido')
df_sinasc['GESTACAO'].value_counts()

C:\Users\Bill_\AppData\Local\Temp\ipykernel_1344\3298576869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinasc['GESTACAO'] = df_sinasc['GESTACAO'].fillna('Não preenchido')


37 a 41 semanas        22243
32 a 36 semanas         2189
Não preenchido          1165
42 semanas e mais        693
28 a 31 semanas          158
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [112]:
# 8) seu código aqui
'''O Apgar possui uma classificação indicando se o bebê passou por asfixia:

Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.
Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.'''
df_sinasc.loc[df_sinasc['APGAR5']>=8, 'ASFIXIA'] = 'normal'
df_sinasc.loc[(df_sinasc['APGAR5']>=6) & (df_sinasc['APGAR5']<=7), 'ASFIXIA'] = 'leve'
df_sinasc.loc[(df_sinasc['APGAR5']>=4) & (df_sinasc['APGAR5']<=5), 'ASFIXIA'] = 'moderada'
df_sinasc.loc[(df_sinasc['APGAR5']>=0) & (df_sinasc['APGAR5']<=3), 'ASFIXIA'] = 'severa'
display(df_sinasc[df_sinasc['ASFIXIA']=='severa'].head(2))
display(df_sinasc[df_sinasc['ASFIXIA']=='moderada'].head(2))
display(df_sinasc[df_sinasc['ASFIXIA']=='leve'].head(2))
display(df_sinasc[df_sinasc['ASFIXIA']=='normal'].head(2))


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,ASFIXIA
20,4,29,5.0,4 a 7 anos,3.0,32 a 36 semanas,Única,3.0,0.0,severa
185,1,24,1.0,8 a 11 anos,0.0,22 a 27 semanas,Única,3.0,3.0,severa


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,ASFIXIA
182,1,30,5.0,8 a 11 anos,2.0,22 a 27 semanas,Única,3.0,4.0,moderada
418,1,14,5.0,4 a 7 anos,0.0,37 a 41 semanas,Única,3.0,5.0,moderada


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,ASFIXIA
14,1,31,1.0,4 a 7 anos,1.0,37 a 41 semanas,Única,4.0,7.0,leve
205,1,28,2.0,4 a 7 anos,2.0,37 a 41 semanas,Única,3.0,6.0,leve


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,ASFIXIA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0,normal


In [118]:
frequencia_por100 = df_sinasc['ASFIXIA'].value_counts(normalize=True) * 100
print(frequencia_por100)
df_sinasc['ASFIXIA'].value_counts()

normal      98.285607
leve         1.190656
severa       0.278824
moderada     0.244913
Name: ASFIXIA, dtype: float64


normal      26085
leve          316
severa         74
moderada       65
Name: ASFIXIA, dtype: int64

In [131]:
# 9) seu código aqui


#df_sinasc.columns = df_sinasc.columns.str.lower().str.replace(' ', '_')
df_sinasc = df_sinasc.applymap(lambda x: str(x).lower().replace(' ', '_'))
df_sinasc


,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,asfixia
0,1,19,5.0,8_a_11_anos,0.0,37_a_41_semanas,única,4.0,10.0,normal
1,1,29,2.0,8_a_11_anos,1.0,37_a_41_semanas,única,4.0,9.0,normal
2,1,37,nan,8_a_11_anos,2.0,37_a_41_semanas,única,4.0,10.0,normal
3,1,30,5.0,12_anos_ou_mais,0.0,37_a_41_semanas,única,3.0,10.0,normal
4,1,30,2.0,8_a_11_anos,1.0,37_a_41_semanas,única,4.0,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12_anos_ou_mais,1.0,32_a_36_semanas,única,4.0,9.0,normal
27024,1,19,1.0,8_a_11_anos,0.0,37_a_41_semanas,única,4.0,9.0,normal
27025,1,24,2.0,8_a_11_anos,0.0,37_a_41_semanas,única,4.0,10.0,normal
27026,1,21,2.0,8_a_11_anos,1.0,32_a_36_semanas,única,4.0,9.0,normal
